# Analysing Damodaran's S&P500 analysis

AIM: check if Damodaran's analysis suggest it is an appropriate time to invest in the S&P500

Aswart Damodaran ([website](https://pages.stern.nyu.edu/~adamodar/New_Home_Page/home.htm)) is a SternU professor.
I watched most of his teeaching materials, including his full valuation class.
He is very analytical in the way he dissects the components that contribute to the value of a company.
Similarly, and this is relevant to this notebook, he decomposes the value of stock expected returns, using the S&P proxy.

Basically, the expected stock return is decomposed into:
- RFR (Risk Free Rate), proxied as the 10Y US Treasury yield that is causally related to the expected USD inflation and therefore also the future stability of United States
- Implied ERP (Equity Risk Premium), which is computed from the current S&P500 price and the expected future earnings. It captures the willingnes of investors to risk over equities to have higher return at the price of possible drawdowns.

He updates this analysis every first day of the month.

Note that this analysis is US-centric, but considering that US is currently 55% of the world market cap (FTSE All World Index, May 2023), and geopolitically the driver of the open-market economy, it is fair to focus on it, also considering the larger availability of data.


In [1]:
import pandas as pd # Note: needs xlrd and openpyxl to be installed to read excel files
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('plotting.backend', 'plotly')

import ssl
ssl._create_default_https_context = ssl._create_unverified_context # Needed to download data

In [2]:
# There are two source of data:
# - yearly data, since 1960
# - monthly data, since September 2008
url_yearly = "https://pages.stern.nyu.edu/~adamodar/pc/datasets/histimpl.xls"
url_monthly = "https://pages.stern.nyu.edu/~adamodar/pc/implprem/ERPbymonth.xlsx"

LAST_YEAR = 2022 # Modify accordingly, to skip the part under the main table
dfy = (
    pd.read_excel(url_yearly, sheet_name="Historical Impl Premiums", skiprows=6, nrows=LAST_YEAR-1959)
)

dfm = (
    pd.read_excel(url_monthly, sheet_name="Historical ERP")
)

/Users/danieleongari/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
pd.to_datetime([120])

DatetimeIndex(['1970-01-01 00:00:00.000000120'], dtype='datetime64[ns]', freq=None)

In [4]:
# Checkpoint to reload original data if I mess up
dfm_orig, dfy_orig = dfm.copy(), dfy.copy()

In [5]:
# Clean data and make column names consistent
dfy, dfm = dfy_orig.copy(), dfm_orig.copy()

dfy = (
    dfy
    .rename(columns={"Year": "Date"})
    .assign(Date=lambda x: pd.to_datetime((x["Date"]+1).astype(str))) # Date refers to the end of the year, so I will add 1 to convert it to the first day of the next year
)
display(dfy)
dfm = (
    dfm
    .rename(columns={"Start of month": "Date"})
    .assign(Date=lambda x: pd.to_datetime(x["Date"]))
)
display(dfm)


,Date,Earnings Yield,Dividend Yield,S&P 500,Earnings*,Dividends*,Dividends + Buybacks,Change in Earnings,Change in Dividends,T.Bill Rate,T.Bond Rate,Bond-Bill,Smoothed Growth,Implied Premium (DDM),Analyst Growth Estimate,Implied ERP (FCFE),Implied Premium (FCFE with sustainable Payout),ERP/Riskfree Rate
0,1961-01-01,0.053400,0.034100,58.11,3.103074,1.981551,NaN,NaN,NaN,0.0266,0.0276,0.0010,0.024484,NaN,NaN,NaN,NaN,NaN
1,1962-01-01,0.047100,0.028500,71.55,3.370005,2.039175,NaN,0.086021,0.029080,0.0213,0.0235,0.0022,0.024051,0.0292,NaN,0.0292,NaN,1.242553
2,1963-01-01,0.058100,0.034000,63.10,3.666110,2.145400,NaN,0.087865,0.052092,0.0273,0.0385,0.0112,0.040496,0.0356,NaN,0.0356,NaN,0.924675
3,1964-01-01,0.055100,0.031300,75.02,4.133602,2.348126,NaN,0.127517,0.094493,0.0312,0.0414,0.0102,0.049635,0.0338,NaN,0.0338,NaN,0.816425
4,1965-01-01,0.056200,0.030500,84.75,4.762950,2.584875,NaN,0.152252,0.100825,0.0354,0.0421,0.0067,0.051323,0.0331,NaN,0.0331,NaN,0.786223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2019-01-01,0.059174,0.021385,2506.85,148.340000,53.610000,136.65,0.187290,0.078021,0.0237,0.0268,0.0031,0.032445,0.0250,0.0412,0.0596,0.0555,2.223881
59,2020-01-01,0.050251,0.018200,3230.78,162.350000,58.800000,150.50,0.094445,0.096810,0.0155,0.0192,0.0037,0.025654,0.0203,0.0396,0.0520,0.0506,2.708333
60,2021-01-01,0.037209,0.015096,3756.07,139.760000,56.700000,127.78,-0.139144,-0.035714,0.0009,0.0093,0.0084,0.007352,0.0165,0.0542,0.0472,0.0494,5.075269
61,2022-01-01,0.043301,0.012421,4766.18,206.380000,59.200000,147.24,0.476674,0.044092,0.0006,0.0151,0.0145,0.017052,0.0172,0.0647,0.0424,0.0490,2.807947


,Date,S&P 500,T.Bond Rate,Ten-year average CF,CF (Trailing 12 month),Normalized CF,Expected growth rate,ERP (T12 m with sustainable payout),ERP (T12m),ERP (Smoothed),ERP (Normalized),ERP (Net Cash Yield),ERP (Covid Adjusted),Expected Return,Notes
0,2008-09-01,1252.00,0.0372,NaN,NaN,NaN,NaN,NaN,0.0422,NaN,NaN,NaN,NaN,0.0794,NaN
1,2008-10-01,1166.00,0.0383,NaN,NaN,NaN,NaN,NaN,0.0451,NaN,NaN,NaN,NaN,0.0834,NaN
2,2008-11-01,969.00,0.0395,NaN,NaN,NaN,NaN,NaN,0.0590,NaN,NaN,NaN,NaN,0.0985,NaN
3,2008-12-01,896.00,0.0292,NaN,NaN,NaN,NaN,NaN,0.0660,NaN,NaN,NaN,NaN,0.0952,NaN
4,2009-01-01,903.00,0.0221,NaN,52.58,NaN,0.0400,NaN,0.0643,NaN,NaN,NaN,NaN,0.0864,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,2024-03-01,5096.00,0.0427,218.02,164.25,153.14,0.0890,0.0418,0.0433,0.0572,0.0404,0.0414,NaN,0.0860,Updated growth rates
187,2024-04-01,5254.00,0.0421,233.26,164.79,159.33,0.0905,0.0413,0.0423,0.0595,0.0411,0.0403,NaN,0.0844,"Updated cash flows, growth rates"
188,2024-05-01,5036.00,0.0469,232.26,164.79,159.33,0.0939,0.0415,0.0440,0.0619,0.0426,0.0423,NaN,0.0909,Updated growth rates
189,2024-06-01,5277.51,0.0438,233.26,164.79,159.33,0.0953,0.0412,0.0427,0.0601,0.0413,0.0406,NaN,0.0865,Updated growth rates


In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dfy["Date"], y=dfy["Implied ERP (FCFE)"], mode="lines+markers", name="Yearly Estimate"))
fig.add_trace(go.Scatter(x=dfm["Date"], y=dfm["ERP (T12m)"], mode="lines", name="Monthly  Estimate"))
fig.update_layout(xaxis_title="Date of Estimation", yaxis_title="Implied ERP", yaxis_type="linear", yaxis_tickformat=',.0%')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=dfy["Date"], y=dfy["T.Bond Rate"], mode="lines+markers", name="RFR Yearly Estimate"))
fig.add_trace(go.Scatter(x=dfm["Date"], y=dfm["T.Bond Rate"], mode="lines", name="RFR Monthly Estimate"))
fig.add_trace(go.Scatter(x=dfy["Date"], y=dfy["T.Bond Rate"]+dfy["Implied ERP (FCFE)"], mode="lines+markers", name="RFR+ERP Yearly Estimate"))
fig.add_trace(go.Scatter(x=dfm["Date"], y=dfm["T.Bond Rate"]+dfm["ERP (T12m)"], mode="lines", name="RFR+ERP  Monthly Estimate"))
fig.update_layout(xaxis_title="Date of Estimation", yaxis_title="T.Bond Rate + ERP", yaxis_type="linear", yaxis_tickformat=',.0%')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=dfy["Date"], y=dfy["Analyst Growth Estimate"], mode="lines+markers", name="Yearly Estimate"))
fig.add_trace(go.Scatter(x=dfm["Date"], y=dfm["Expected growth rate"], mode="lines", name="Monthly  Estimate"))
fig.update_layout(xaxis_title="Date of Estimation", yaxis_title="Growth estimate", yaxis_type="linear", yaxis_tickformat=',.0%')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=dfy["Date"], y=dfy["S&P 500"], mode="lines+markers", name="Yearly Estimate"))
fig.add_trace(go.Scatter(x=dfm["Date"], y=dfm["S&P 500"], mode="lines", name="Monthly  Estimate"))
fig.update_layout(xaxis_title="Date of Estimation", yaxis_title="S&P 500 price", yaxis_type="log")
fig.show()



In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=dfy["Analyst Growth Estimate"], y=dfy["Implied ERP (FCFE)"], 
    hovertext=[ f"Date: {x.date()}" for x in dfy["Date"]], 
    mode="markers", name="Yearly Estimate"))
fig.add_trace(go.Scatter(x=dfm["Expected growth rate"], y=dfm["ERP (T12m)"], 
    hovertext=[ f"Date: {x.date()}" for x in dfm["Date"]], 
    mode="markers", name="Monthly  Estimate"))
fig.add_trace(go.Scatter(x=dfm.iloc[[-1]]["Expected growth rate"], y=dfm.iloc[[-1]]["ERP (T12m)"], 
    hovertext=[ f"Date: {x.date()}" for x in dfm.iloc[[-1]]["Date"]],
    marker_size=20, marker_opacity=0.5, 
    mode="markers", name=f"Now - {dfm.iloc[-1]['Date'].date()}"))
fig.update_layout(
    xaxis_title="Growth Estimate", yaxis_title="Equity Risk Premium",
    xaxis_tickformat=',.0%', yaxis_tickformat=',.0%',
    width=1000, height=800)
fig.show()

## Conclusions
- Independently from the RFR (10-years US Treasury yield) which is a proxy for expected growth+inflation we can use the ERP (Equity Risk Premium) as a proxy for the market greed/fear
- When the ERP is high there is a lot of fear, e.g., the market expect oscillations in the near-future
- When the Growth Estimate is high, typically the market is overexcited, and the growth will mean-revert to some more reasonable long-term trends
- This last statement is more controversial because the analyst may already expect a low groewth after a bull run, still underestimating the downturn of the index: this is the case of low-GrowthEstimate in late 2019, when the analyst were conservative in projecting a lower growth after a booming year. Maybe not a good moment to enter the market, despite the already-low estimated growth expectation.
- As investors, if we assume that there won't be major market disruptions in the future, we want to enter the market when the ERP is high and the Growth Estimate is low (assuming analyst are already conservative), therefore we want to be in the top-left quadrant of the GrowthEst/ERP plot
- We are currently (May 2023) in a mildly good condition to enter the market, with below-average market growth expectation and quite average market fear/greed (considering the ERP). We are not in extreme condition to conclude the market is a bargain nor that it is overpriced.

## Follow-up
- Add the future 1-5-10 years growth, to check if the growth estimate was legit
- Include other macro indicators, to understand when the analyst are too optimistic/pessimistic in their Growth Estimate: however this is not easy, as it is a recursive evaluation, i.e., using macro indicators to evaluate the analyst that are evaluating the macro indicators.